In [1]:
libs <- c("R6","testit","optparse","magrittr","dplyr","tibble","readr",
          "reshape2","fossil","tidyr","purrr", "data.table", "Rcpp", "RcppArmadillo")
y <- lapply(libs, require, character.only = TRUE)
assert("All packages loaded correctly", all(unlist(y))); rm(libs); rm(y)

cat(paste0("||", paste0(rep("-", 27), collapse = ""), " (4/8) Average distances collection ",
           paste0(rep("-", 28), collapse = ""), "||\nStarted process at: ", Sys.time()))

stopwatch <- list("start_time" = as.character.POSIXt(Sys.time()), "end_time" = NULL)

Loading required package: R6

Loading required package: testit

Loading required package: optparse

Loading required package: magrittr

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tibble

Loading required package: readr

Loading required package: reshape2

Loading required package: fossil

Loading required package: sp

Loading required package: maps

Loading required package: shapefiles

Loading required package: foreign


Attaching package: ‘shapefiles’


The following objects are masked from ‘package:foreign’:

    read.dbf, write.dbf


Loading required package: tidyr


Attaching package: ‘tidyr’


The following object is masked from ‘package:reshape2’:

    smiths


The following object is masked from ‘package:magrittr’:

    extract


Loading required package: purrr


A

||--------------------------- (4/8) Average distances collection ----------------------------||
Started process at: 2022-05-10 16:38:21

In [3]:
# Current working directory should be Metrics-CGM-ECC/
files <- c("ECC/classes_ecc.R", "ECC/ecc_functions.R", "ECC/dist_functions.R")
invisible(sapply(files, source)); rm(files)

paste0("Sourcing required functions, reading in input data")

#sourceCpp("epicohversions.cpp")
source("arguments.R")

[1] "Sourcing required functions, reading in input data"

In [4]:
# dm <- dms[["temp"]]; cnames <- "Date"
groupXAvgDists <- function(dm, groupX, assignments, strain_data, cnames, metadata) {
  clustersX <- pull(groupX, 1)

  avgdists <- lapply(clustersX, function(x) {
    cx <- assignments[Hx == x]

    num_unique_cases <- strain_data[Strain %in% cx$Strain] %>%
      select(all_of(cnames)) %>% unique() %>% nrow()
    if (num_unique_cases == 1) {
      return(0)
    }else {
      dr_counts <- cx %>% group_by(dr) %>% summarise(n_drs = n()) %>% as.data.table()
      dmx <- dm[rownames(dm) %in% dr_counts$dr, colnames(dm) %in% dr_counts$dr,drop=FALSE]
      
      dr_counts <- dr_counts[match(rownames(dmx), dr_counts$dr)]
      dmxcols <- sweep(dmx, MARGIN = 2, dr_counts$n_drs, `*`)
      dmxrest <- sweep(dmxcols, MARGIN = 1, dr_counts$n_drs, `*`)
      dmxrest <- dmxrest[upper.tri(dmxrest)]
      
      allstrains <- matrix(nrow = sum(dr_counts$n_drs), ncol = sum(dr_counts$n_drs))
      avg_dist <- sum(dmxrest)/length(allstrains[upper.tri(allstrains)])
      return(avg_dist)
    }
    # if (is.na(avg_dist)) {print(x)}
  }) %>% unlist() %>% data.table(Hx = clustersX, AvgDist = .)
  assert("No NA avg dists", nrow(avgdists[is.na(AvgDist)]) == 0)
  return(avgdists)
}

In [5]:
paste0("Processing input data, formatting ...")

hx <- strsplit(as.character(params$th[2]), split = ",") %>% unlist() %>% tibble(h = ., th = paste0("T", .))

fdata <- readRDS(arg$tpn)$new_cols %>% column_to_rownames("Strain")
tp2 <- Timepoint$new(arg$tpn, "tp2", fdata)$Process(hx)$listHeights(hx)

m <- read_tsv(arg$metadata) %>% processedStrains()
metadata <- m$strain_data %>% as.data.table()

if (params$int_type[2] == "multiset") {
  interval <- "Multiset"
}else if (params$int_type[2] == "monthly") {
  interval <- "YearMonth"
}else if (params$int_type[2] == "weekly") {
  interval <- "YearWeek"
}

[1] "Processing input data, formatting ..."

Rows: 111 Columns: 14
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (7): Strain, Source, Country, Province, City, YearMonth, YearWeek
dbl  (6): Latitude, Longitude, Day, Month, Year, TP2
date (1): Date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [6]:
clustersets <- file.path("../intermediate_data", params$int_type[2], "clustersets.Rds") %>% readRDS(.)
interval_list <- names(clustersets); rm(clustersets)
basedir <- file.path("../intermediate_data", params$int_type[2], "avgdists")
parts <- file.path("../intermediate_data", params$int_type[2], "TPN", "parts.Rds") %>% readRDS()
groups <- names(parts$results)

tpnfiles <- lapply(names(parts$results), function(x) parts$results[[x]] %>% add_column(f = x) %>% 
                     select(-n)) %>% bind_rows() %>% as.data.table() %>% set_colnames(c("Hx", "f"))

strain_data <- as.data.table(m$strain_data)

In [7]:
typing_data <- lapply(1:length(interval_list), function(i) {
  n1 <- as.character(interval_list[i])
  tpkstrains <- metadata[get(interval) <= n1]$Strain
  dfz <- tp2$filedata %>% rownames_to_column("isolate") %>%
    select(isolate, all_of(hx$h)) %>%
    filter(isolate %in% tpkstrains) %>% column_to_rownames("isolate")
  dfz[,hx$h[1],drop=FALSE] %>% set_colnames(hx$th[1])
}) %>% set_names(as.character(interval_list))

In [10]:
paste0("Collecting average distances (for each cluster) at ...")

for (tdx in names(typing_data)) {
  outputDetails(paste0("TP", tdx, ":"), newcat = TRUE)
  
  td <- typing_data[[tdx]] %>% rownames_to_column("Strain") %>% as.data.table()
  pd1 <- parts$drs[Strain %in% td$Strain] %>% set_colnames(c("Strain", "Hx", "dr")) %>% 
    inner_join(., tpnfiles, by = "Hx") %>% arrange(f)
  pd2 <- pd1 %>% group_by(Hx) %>% summarise(n = n()) %>% left_join(pd1, ., by = "Hx") %>% 
    select(-Strain) %>% unique()
  
  assignments <- inner_join(m$dr_matches, td, by = "Strain") %>% 
    set_colnames(c("Strain", "dr", "Hx")) %>% as.data.table()
  
  sizes <- td %>% set_colnames(c("Strain", "Hx")) %>% group_by(Hx) %>% summarise(Size = n())
  
  pb <- txtProgressBar(min = 0, max = length(groups), initial = 0, style = 3)
  avg_dists <- lapply(1:length(groups), function(i) {
    setTxtProgressBar(pb, i)
    # print(i)
    x_i <- groups[i]
    dms <- paste0("group", x_i, ".Rds") %>% 
      file.path("../intermediate_data", params$int_type[2], "TPN", "dists", .) %>% readRDS()
    groupX <- pd2[f == x_i] %>% select(Hx, n) %>% unique()

    file.path()
    if (nrow(groupX) > 0) {
      temp_dists <- dms[["temp"]] %>% 
        groupXAvgDists(., groupX, assignments, strain_data, "Date", metadata) %>% 
        rename(Temp.Avg.Dist = AvgDist)
      
      geo_dists <- dms[["geo"]] %>% 
        groupXAvgDists(., groupX, assignments, strain_data, c("Longitude", "Latitude"), metadata) %>% 
        rename(Geo.Avg.Dist = AvgDist)  
      
      return(merge.data.table(temp_dists, geo_dists))
    }
  }) %>% bind_rows() %>% inner_join(., sizes, by = "Hx")
  close(pb)
  
  paste0("   Checking that only clusters with just one unique info pair have average distance 0")
  assert("No NA avg geo distances", !any(is.na(avg_dists$Geo.Avg.Dist)))
  assert("No NA avg temp distances", !any(is.na(avg_dists$Temp.Avg.Dist)))
  
  # Check that clusters with the average pairwise distance being 0 have only one
  # unique date / coordinate pair, as is required to get a distance result of 0
  renamed_tping <- td %>% set_colnames(c("Strain", "Hx"))
  
  temp_zero_clusters <- avg_dists[Temp.Avg.Dist == 0]$Hx
  for (x in temp_zero_clusters) {
    y <- renamed_tping[Hx == x] %>% pull(Strain)
    unidate <- strain_data[Strain %in% y, Date] %>% unique() %>% length()
    assert(paste0("Cluster ", x, " has only one unique date"), unidate == 1)
  }
  
  geo_zero_clusters <- avg_dists[Geo.Avg.Dist == 0]$Hx
  for (x in geo_zero_clusters) {
    y <- renamed_tping[Hx == x] %>% pull(Strain)
    uniloc <- strain_data[Strain %in% y] %>% select(Latitude, Longitude) %>% unique() %>% nrow()
    assert(paste0("Cluster ", x, " has only one unique coordinate pair"), uniloc == 1)
  }

  colnames(avg_dists)[which(colnames(avg_dists) == "Hx")] <- hx$th
  
  # Collects raw cluster averages as well
  curr_data <- left_join(td, strain_data, by = "Strain") %>% 
    set_colnames(gsub(hx$th, "heightx", colnames(.)))
  clusters <- curr_data %>% pull(heightx) %>% unique()
  
  reg_avgs <- lapply(clusters, function(c_i) {
    dfx <- curr_data[heightx == c_i]
    data.table(c_i, mean(dfx$Date), mean(dfx$Longitude), mean(dfx$Latitude))
  }) %>% bind_rows() %>% 
    set_colnames(c(hx$th, "Avg.Date", "Avg.Longitude", "Avg.Latitude"))
  
  merge.data.table(avg_dists, reg_avgs) %>% 
    set_colnames(gsub("Size", paste0(hx$th, "_Size"), colnames(.))) %>% 
    add_column(TP = tdx, .before = 1) %>% 
    saveRDS(., file.path(basedir, paste0("TP", tdx, ".Rds")))
}

[1] "Collecting average distances (for each cluster) at ..."

TP2020-03:


TP2020-03:



  |======================================================================| 100%
TP2020-04:


TP2020-04:



...

  |======================================================================| 100%
TP2020-12:


TP2020-12:



  |======================================================================| 100%
TP2020-13:


TP2020-13:



  |======================================================================| 100%


In [11]:
all_avg_dists <- lapply(names(typing_data), function(tdx) {
  readRDS(file.path(basedir, paste0("TP", tdx, ".Rds")))
}) %>% set_names(names(typing_data)) %>% bind_rows()

if (params$int_type[2] == "multiset") {
  res_file <- gsub("-", "", params$divs[2]) %>% gsub(",", "-", .) %>% 
    paste0("AVGS-",.,  ".Rds") %>% 
    file.path(file.path("../results", params$int_type[2]), .)
}else {
  res_file <- file.path("../results", params$int_type[2], "AVGS-intervals.Rds")
}

paste0("Merging average distance results and saving to '", res_file, "'")
saveRDS(all_avg_dists, res_file)
stopwatch[["end_time"]] <- as.character.POSIXt(Sys.time())
timeTaken(pt = "average distances collection", stopwatch)
cat(paste0("||", paste0(rep("-", 27), collapse = ""), " End of average distances collection ",
           paste0(rep("-", 27), collapse = ""), "||"))

[1] "Merging average distance results and saving to '../results/weekly/AVGS-intervals.Rds'"

[1] "\nThe average distances collection process took 2 minute(s) and 49 second(s)."

||--------------------------- End of average distances collection ---------------------------||